## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

In [1]:
import numpy as np

ch_ideal = ["Pz", "PO3", "PO5", "PO4", "PO6", "POz", "O1", "Oz", "O2"] #canais ideais
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]


data = np.load("../../datasets/beta/data2.npy")


labels = np.load("../../datasets/beta/labels2.npy")

data.shape,labels.shape

((160, 64, 750), (160,))

In [2]:
import mne

n_channels = 64
sfreq = 250
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=(['eeg'] * len(ch_names)))

ev = {str(value): index  for index, value in enumerate(sorted(set(labels)))}

labels.max(), labels.min()

(15.8, 8.0)

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)


mne_data = mne.EpochsArray(data, info, events, event_id=ev)
filtered_mne_data = mne.EpochsArray(data, info, events, event_id=ev).filter(7.8, 16.0)
mne_data.get_data().shape

Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 7.8 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.80
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.80 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    0.4s
[Parallel(n_job

(160, 64, 750)

In [4]:
import mne
import numpy as np

# define a baseline
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

sr = 250
target_frequencies = np.arange(8, 16, 0.2) # define as frequências alvo

target_amplitudes = []

base_powers = []
rest_powers = []
snrs = []

for channel_data in mne_data.get_data():
    target_per_channel = []  # guarda as amplitudes para cada frequência alvo

    base_power_channel = []   # guarda o base_power para cada trial
    rest_power_channel = []  # guarda o rest_power para cada trial

    for sign in channel_data:
        fft_result = np.fft.fft(sign)  # FFT
        psd = np.abs(fft_result) ** 2  # PSD    
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)  # frequencias

        target_amplitudes_trial = []

        for target_frequency in target_frequencies:
            index = np.argmin(np.abs(frequencies - target_frequency))  # acha o índice da frequência alvo
            amplitude = np.sqrt(psd[index])  # calcula a amplitude da frequência alvo
            target_amplitudes_trial.append(amplitude)  # guarda a amplitude da frequência alvo

        target_per_channel.append(target_amplitudes_trial)  # guarda as amplitudes para cada frequência alvo

        base_power = np.mean(psd[base_start:base_end]) # calcula o base_power
        rest_power = np.mean(psd[rest_start:rest_end]) # calcula o rest_power

        base_power_channel.append(base_power) # guarda o base_power para cada trial
        rest_power_channel.append(rest_power) 


    snr = np.mean(base_power_channel) + np.mean(rest_power_channel) # calcula o snr

    base_powers.append(base_power_channel)
    rest_powers.append(rest_power_channel)
    snrs.append(snr)

    target_amplitudes.append(target_per_channel)  # guarda as amplitudes para cada frequência alvo


target_amplitudes = np.array(target_amplitudes)  # transforma em numpy array
base_powers = np.array(base_powers)
rest_powers = np.array(rest_powers)
snrs = np.array(snrs)

snrs

array([ 34254.95173393,  29043.47729528,  27617.12260873,  36635.40100522,
        35827.14509781,  32329.80178018,  34410.5735515 ,  34515.60391904,
        27721.05542327,  33373.41590558,  28185.94595284,  32820.54728614,
        31836.71119898,  30179.19691795,  32185.71427697,  26173.01812723,
        29865.45928936,  37986.3515825 ,  34162.96889245,  41848.39179112,
        33522.94167244,  37515.57297451,  28917.13458999,  34899.69628569,
        31015.82396003,  33891.54548758,  33661.20404166,  31563.14510089,
        30697.41093828,  37299.3793779 ,  34699.66997813,  53382.94099808,
        34762.4225378 ,  34891.99421989,  34829.39121345,  28709.38604112,
        68402.33150205,  37375.68486614,  36245.60307063,  51493.82464981,
        29712.27363049,  34079.973482  ,  34928.66938484,  32717.68285686,
        30770.37077287,  26976.50698312,  33077.26244184,  39021.03985767,
        22257.51002307,  30746.71519765,  30158.82108184,  33263.88032245,
        32469.42694901,  

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [5]:
snr = np.min(snrs) # pega o menor snr
target_amplitudes_adjusted = np.abs(target_amplitudes - snr) #ajuste da amplitude do sinal
 
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / snr) # razao sinal ruido de banda estreita
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted) # potencia total
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power) # razao sinal ruido de banda larga
print(wide_band_SNR.shape)



data = np.array([narrow_band_SNR, wide_band_SNR]) # array com as duas razoes sinal ruido
print(data.shape)
data = data.swapaxes(0, 1) # troca de eixos
print(data.shape)
data = data.swapaxes(1, 2) # troca de eixos
print(data.shape)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3]) # reshape

#salva os dados


np.save("../../datasets/beta/data2_filtered.npy", data)

(160, 64, 40)
(160, 64, 40)
(2, 160, 64, 40)
(160, 2, 64, 40)
(160, 64, 2, 40)


Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.